# Monte Carlo simulation for cyber threats portfolio

This python script uses a Monte Carlo simulation to calculate the total cyber-value-at-risk for a portfolio of cyber threats.

In [ ]:
#Import the libraries
import pandas as pd
import random
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Define a Pandas Dataframe with the cyber events, probability of occurrence (the likelihood) and the impact in dollars
# in a upper and lower range

data = {'Event': ['Phishing Attack', 'Ransomware Attack', 'DDoS Attack', 'Data Breach'],
        'Probability': [0.5, 0.2, 0.05, 0.35],
        'Lower': [10000, 50000, 100000, 250000],
        'Upper': [50000, 100000, 250000, 500000]}

cyber_threats = pd.DataFrame(data)



Understanding the functions and what they do:

1 - is_attack_successful(probability) : This function generates a random number between 0 and 1 and checks if it is less than or equal to the probability passed as an argument. If it is, the attack is considered successful and the function returns True. If not, it returns False.

2 - calculate_loss(lower, upper) : This function generates a random number between the lower and upper bounds passed as arguments and returns it.

3 - simulate_risk_portfolio(cyber_threats) : This function iterates over each risk in the cyber_threats DataFrame and checks if the attack is successful using the is_attack_successful function. If it is, it calculates the loss using the calculate_loss function and adds it to the total loss. It returns the total loss.

4 - monte_carlo_simulation(cyber_threats, iterations) : This function runs the simulation by calling the simulate_risk_portfolio function for the number of iterations passed as an argument. It stores the result of each iteration in the losses_per_year list and returns it.

References: 

https://www.dataskunkworks.com/latest-posts/building-a-quantitative-cyber-risk-model-in-python

https://ora.ox.ac.uk/objects/uuid:e22b206e-5c7b-4298-9519-dcd37fd21bce/download_file?safe_filename=Erola_et_al_2021_a_system_to.pdf&type_of_work=Journal+article


In [ ]:
# Setup the functions for the Monte Carlo simulation

def is_attack_successful(probability):
    return random.random() <= probability

def calculate_loss(lower, upper):
    return random.uniform(lower, upper)

def simulate_risk_portfolio(cyber_threats):
    total_loss = 0
    for risk in cyber_threats.itertuples():
        if is_attack_successful(risk.Probability):
            total_loss += calculate_loss(risk.Lower, risk.Upper)
    return total_loss

iterations = 1000
def monte_carlo_simulation(cyber_threats, iterations):
    losses_per_year = []
    for i in range(iterations):
        loss = simulate_risk_portfolio(cyber_threats)
        losses_per_year.append(loss)
    return losses_per_year

losses_per_year = monte_carlo_simulation(cyber_threats, iterations)


In [ ]:
# Visualise the data

# Plot the yearly losses
plt.plot(losses_per_year)
plt.xlabel("Iterations")
plt.ylabel("Yearly Losses")
plt.show()

# Plot the loss exceedance curve
sorted_losses = sorted(losses_per_year)
p = 1 - np.arange(1, len(losses_per_year) + 1) / len(losses_per_year)
plt.plot(sorted_losses, p)
plt.xlabel("Losses($)")
plt.ylabel("Probability of exceedance")
plt.show()

# Assess convergence

Assessing convergence is an important step in any Monte Carlo simulation, as it helps to ensure that the results of the simulation are reliable and accurate. It seems we need a few more iterations than anticipated.

In [ ]:
import numpy as np

# run the simulation for a large number of iterations
num_iterations = 10000
losses_per_year = monte_carlo_simulation(cyber_threats, num_iterations)

# Divide the total number of iterations into blocks
block_size = 100
num_blocks = int(num_iterations/block_size)

# calculate the variance of the results for each block
variances = []
for i in range(num_blocks):
    start = i*block_size
    end = (i+1)*block_size
    block_losses = losses_per_year[start:end]
    variance = np.var(block_losses)
    variances.append(variance)

# Plot the variances of each block
plt.figure(figsize=(10,5))
plt.plot(range(num_blocks), variances)
plt.xlabel("Block number")
plt.ylabel("Variance of losses")
plt.show()

# Plot nicer graphs

In [ ]:
# Set the figure size
plt.figure(figsize=(10,5))
# Set the plot style
plt.style.use('ggplot')

# Add grid lines
plt.grid(True)

# Plot the annualized losses over the years
plt.plot(losses_per_year)

# Customize the axis labels
plt.xlabel("Iterations", fontsize=12, color='black', fontweight='bold')
plt.ylabel("Yearly losses($)", fontsize=12, color='black', fontweight='bold')

# Add a title
plt.title("Yearly losses", fontsize=14, fontweight='bold')

In [ ]:
sorted_losses = sorted(losses_per_year)
p = 1 - np.arange(1, len(losses_per_year) + 1) / len(losses_per_year)

# Set the figure size
plt.figure(figsize=(10,5))
# Set the plot style
plt.style.use('ggplot')

# Add grid lines
plt.grid(True)

# Plot the annualized losses over the years
plt.plot(sorted_losses, p)

# Customize the axis labels
plt.xlabel("Losses($)", fontsize=12, color='black', fontweight='bold')
plt.ylabel("Probability of exceedance", fontsize=12, color='black', fontweight='bold')

# Add a title
plt.title("loss exceedance curve", fontsize=14, fontweight='bold')


In [ ]:
# Plot the histogram of losses
plt.figure(figsize=(10,5))
plt.hist(losses_per_year, bins = 50)
plt.xlabel("Losses")
plt.ylabel("Frequency")
plt.show()

# Descriptive statistics

In [ ]:
# Descriptive statistics

mean = np.mean(losses_per_year)
print("Mean loss: ", mean)

std_dev = np.std(losses_per_year)
print("Standard deviation of losses: ", std_dev)

variance = np.var(losses_per_year)
print("Variance of losses: ", variance)

min_loss = np.min(losses_per_year)
print("Minimum loss: ", min_loss)

max_loss = np.max(losses_per_year)
print("Maximum loss: ", max_loss)

# Annualized loss

In [ ]:
# Create a new column in the cyber_threats DataFrame to store the annualized loss for each event
cyber_threats["Annualized Loss"] = 0
iterations = 10000
time_horizon = 10

def attack_loss_amount(lower, upper):
    loss = random.uniform(lower, upper)
    return loss

# Iterate over each event
for event in cyber_threats.itertuples():
    event_losses = []
    # Run the Monte Carlo simulation for the current event
    for i in range(iterations):
        loss = attack_loss_amount(event.Lower, event.Upper)
        event_losses.append(loss)
    # Calculate the annualized loss for the current event
    annualized_loss = sum(event_losses) / (time_horizon * iterations)
    # Update the "Annualized Loss" column for the current event
    cyber_threats.loc[cyber_threats["Event"] == event.Event, "Annualized Loss"] = annualized_loss

print(cyber_threats)
